In [1]:
#Colab: 04_train_transfer.ipynb
from google.colab import drive
drive.mount('/content/drive')
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pickle
PROJECT_ROOT='/content/drive/MyDrive/aerial_project'
CLASSIFICATION_DIR=os.path.join(PROJECT_ROOT, 'classification_dataset')
DETECTION_DIR=os.path.join(PROJECT_ROOT, 'object_detection_Dataset')
DATA_DIR=os.path.join(PROJECT_ROOT, 'data')
SAVED_MODELS=os.path.join(PROJECT_ROOT, 'saved_models')
RESULTS_DIR=os.path.join(PROJECT_ROOT, 'results')
#create important folders if they don't exist
os.makedirs(SAVED_MODELS, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
print('PROJECT_ROOT:', PROJECT_ROOT)
print('CLASSIFICATION_DIR exists:', os.path.exists(CLASSIFICATION_DIR))
print('DETECTION_DIR exists:', os.path.exists(DETECTION_DIR))

#Data augmentation setup (copied from nb. 02)
data_augmentation=tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.12),
    layers.RandomZoom(0.12),
    layers.RandomTranslation(0.08, 0.08),
], name='data_augmentation')

def build_efficientnet(input_shape=(224,224,3), num_classes=2, freeze_until=100):
  base=EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
  base.trainable=True
  for layer in base.layers[:freeze_until]:
    layer.trainable=False

  inputs=layers.Input(shape=input_shape)
  x=layers.Rescaling(1./255)(inputs)
  x=data_augmentation(x)
  x=base(x, training=False)
  x=layers.GlobalAveragePooling2D()(x)
  x=layers.BatchNormalization()(x)
  x=layers.Dropout(0.4)(x)
  x=layers.Dense(256, activation='relu')(x)
  x=layers.Dropout(0.3)(x)
  outputs=layers.Dense(num_classes, activation='softmax')(x)
  return models.Model(inputs, outputs, name='efficientnet_b0')

model=build_efficientnet()
model.summary()
model.compile(
optimizer=tf.keras.optimizers.Adam(1e-4),
loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=['accuracy'])

callbacks=[
tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
tf.keras.callbacks.ModelCheckpoint(os.path.join(SAVED_MODELS,'best_efficientnet.h5'), save_best_only=True, monitor='val_loss'),
tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)]

#Dataset Loading and Class Weight Setup (copied from nb. 03)
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.utils import class_weight
import numpy as np

IMG_SIZE=(224, 224)
BATCH_SIZE=32
#train, val, test datasets
train_ds=tf.keras.utils.image_dataset_from_directory(
    os.path.join(CLASSIFICATION_DIR, 'train'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

#saving the class names(so evaluation and/or streamlit use the same mapping)
import json
class_names=train_ds.class_names
print("Class names (saved):", class_names)
with open(os.path.join(SAVED_MODELS, 'class_names.json'), 'w') as f:
    json.dump(class_names, f)

val_ds=tf.keras.utils.image_dataset_from_directory(
    os.path.join(CLASSIFICATION_DIR, 'valid'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

test_ds=tf.keras.utils.image_dataset_from_directory(
    os.path.join(CLASSIFICATION_DIR, 'test'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

#class names
class_names=train_ds.class_names
print("Classes:", class_names)

#computing class weights (to handle imbalance)
labels=np.concatenate([y for x, y in train_ds], axis=0)
class_weights_vals=class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weights=dict(enumerate(class_weights_vals))
print("Class Weights:", class_weights)

# Prefetch for performance
AUTOTUNE=tf.data.AUTOTUNE
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds=test_ds.cache().prefetch(buffer_size=AUTOTUNE)

history=model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=callbacks, class_weight=class_weights)
#optional fine-tuning: unfreezing the last N layers and train with lower lr
fine_tune_at=len(model.layers)-50
for layer in model.layers[fine_tune_at:]:
  layer.trainable=True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=['accuracy'])
history_finetune=model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks, class_weight=class_weights)

#saving history and model
import pickle
with open(os.path.join(SAVED_MODELS,'history_efficientnet.pkl'),'wb') as f:
  pickle.dump({'initial': history.history, 'finetune': history_finetune.history}, f)
model.save(os.path.join(SAVED_MODELS,'best_efficientnet_full.h5'))

Mounted at /content/drive
PROJECT_ROOT: /content/drive/MyDrive/aerial_project
CLASSIFICATION_DIR exists: True
DETECTION_DIR exists: True
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "efficientnet_b0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,383,141 (16.72 MB)

 Trainable params: 4,171,358 (15.91 MB)

...

Found 2662 files belonging to 2 classes.
Class names (saved): ['bird', 'drone']
Found 442 files belonging to 2 classes.
Found 215 files belonging to 2 classes.
Classes: ['bird', 'drone']
Class Weights: {0: np.float64(0.9413012729844413), 1: np.float64(1.0665064102564104)}
Epoch 1/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.5832 - loss: 1.0378

84/84 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.5835 - loss: 1.0369 - val_accuracy: 0.4910 - val_loss: 0.6993 - learning_rate: 1.0000e-04
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 167ms/step - accuracy: 0.6533 - loss: 0.7964 - val_accuracy: 0.4706 - val_loss: 0.7088 - learning_rate: 1.0000e-04
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.6688 - loss: 0.7929 - val_accuracy: 0.4977 - val_loss: 0.8590 - learning_rate: 1.0000e-04
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.6869 - loss: 0.7229 - val_accuracy: 0.5973 - val_loss: 0.7230 - learning_rate: 1.0000e-04
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.7168 - loss: 0.6143

84/84 ━━━━━━━━━━━━━━━━━━━━ 16s 194ms/step - accuracy: 0.7166 - loss: 0.6146 - val_accuracy: 0.7760 - val_loss: 0.4606 - learning_rate: 5.0000e-05
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.7141 - loss: 0.6203

84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 181ms/step - accuracy: 0.7143 - loss: 0.6199 - val_accuracy: 0.7692 - val_loss: 0.4558 - learning_rate: 5.0000e-05
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 165ms/step - accuracy: 0.7393 - loss: 0.5424 - val_accuracy: 0.6968 - val_loss: 0.5479 - learning_rate: 5.0000e-05
Epoch 8/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 20s 163ms/step - accuracy: 0.7489 - loss: 0.5173 - val_accuracy: 0.6946 - val_loss: 0.5542 - learning_rate: 5.0000e-05
Epoch 9/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.7392 - loss: 0.5330 - val_accuracy: 0.7172 - val_loss: 0.4865 - learning_rate: 5.0000e-05
Epoch 10/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 165ms/step - accuracy: 0.7592 - loss: 0.4996 - val_accuracy: 0.7308 - val_loss: 0.5024 - learning_rate: 2.5000e-05
Epoch 11/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.7423 - loss: 0.5255 - val_accuracy: 0.7059 - val_loss: 0.5674 - learning_rate: 2.5000e-05
Epoch 12/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.

84/84 ━━━━━━━━━━━━━━━━━━━━ 16s 195ms/step - accuracy: 0.7284 - loss: 0.5243 - val_accuracy: 0.7964 - val_loss: 0.4425 - learning_rate: 2.5000e-05
Epoch 13/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 166ms/step - accuracy: 0.7508 - loss: 0.4924 - val_accuracy: 0.7715 - val_loss: 0.4846 - learning_rate: 2.5000e-05
Epoch 14/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.7553 - loss: 0.4840 - val_accuracy: 0.6742 - val_loss: 0.6473 - learning_rate: 2.5000e-05
Epoch 15/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.7663 - loss: 0.4670 - val_accuracy: 0.7851 - val_loss: 0.4926 - learning_rate: 2.5000e-05
Epoch 16/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy: 0.7531 - loss: 0.5077 - val_accuracy: 0.7624 - val_loss: 0.5635 - learning_rate: 1.2500e-05
Epoch 17/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 20s 164ms/step - accuracy: 0.7744 - loss: 0.4501 - val_accuracy: 0.7647 - val_loss: 0.5064 - learning_rate: 1.2500e-05
Epoch 18/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 14s 164ms/step - accuracy

In [2]:
#Manual evaluation metrics (after training)
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
#Get predictions
y_true=np.concatenate([y for x, y in test_ds], axis=0)
y_pred_probs=model.predict(test_ds)
y_pred=np.argmax(y_pred_probs, axis=1)
#Classification report
print(classification_report(y_true, y_pred, target_names=class_names))
#Confusion matrix
cm=confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 470ms/step
              precision    recall  f1-score   support

        bird       0.91      0.60      0.73       121
       drone       0.64      0.93      0.76        94

    accuracy                           0.74       215
   macro avg       0.78      0.76      0.74       215
weighted avg       0.80      0.74      0.74       215

Confusion Matrix:
 [[73 48]
 [ 7 87]]
